![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
# QuantBook Analysis Tool
# For more information see https://www.quantconnect.com/docs/research/overview
qb = QuantBook()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

start_day = datetime(2019, 1, 1)
end_day = datetime(2020, 1, 1)

ko = qb.AddEquity('KO')
pep = qb.AddEquity('PEP')



In [8]:
history_ko = qb.History(ko.Symbol, start_day, end_day, Resolution.Hour)
history_pep = qb.History(pep.Symbol, start_day, end_day, Resolution.Hour)

history = pd.concat([history_ko, history_pep], axis=0)
history = history['close'].unstack(level=0)
history = history.dropna(axis=1)

df = history
df

In [3]:
S1 = df['KO']
S2 = df['PEP']


In [9]:
#vi tilføjer en konstant term til vores regression, så vi kan fitte. se https://stackoverflow.com/questions/41404817/statsmodels-add-constant-for-ols-intercept-what-is-this-actually-doing
S1 = sm.add_constant(S1)
results = sm.OLS(S2, S1).fit()
S1 = S1['KO']
b = results.params['KO']
spread = S2-b *S1
print(S2)
df['spread'] = S2-b *S1


spread.plot(figsize=(20, 10))
plt.axhline(spread.mean(), color='black')
plt.legend(['spread'], prop={'size': 20})





In [5]:
#Get the returns nice and 
def zscore(series):
    return (series -series.mean()) / np.std(series)
    

In [6]:
score = zscore(spread)
onestd = 1
minusstd = -1
score.plot(figsize=(20, 10))
plt.axhline(score.mean(), color = 'black')
plt.axhline(onestd, color = 'green')
plt.axhline(minusstd, color = 'red')
plt.axhline(2, color='blue')
plt.axhline(-2, color='blue')

In [7]:
plt.figure(figsize=(20, 16))
plt.plot(df['KO'])
plt.plot(df['PEP'])